In [1]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
#IMPORTING THE DATASET
df = pd.read_csv('train.csv')

#SACAR COLUMNAS INNECESARIAS
df = df.drop(['PassengerId', 'Ticket', 'Cabin'], axis = 1)

#CHEKEAR VALOR MAS REPETIDO Y RELLENAR na
df = df.fillna({"Embarked": "S"})

#SACAR TITULOS DE LOS NOMBRES
s = df['Name'].apply(lambda st: st[st.find(", ")+1:st.find(". ")]).rename('Title')

#HACER JOIN DE COLUMNA TITULOS Y BORRAR COLUMNA NOMBRE
df = pd.concat([df, s], axis = 1).drop(['Name'], axis = 1)

#TRANSFORMAR Pclass A TEXTO Y CREAR COLUMNA TP(Title+Pclass) PARA APLICAR SPLIT+APPLY+COMBINE PARA RELLENAR 
#na EN Age
df['Pclass'] = df['Pclass'].astype(str)
df['TP'] = df['Title'] + df['Pclass']
#AGRUPA POR TP Y SACA EL AVG DE Age
grouped = df.groupby('TP').agg(avg = pd.NamedAgg(column = 'Age', aggfunc = 'mean')).reset_index()
#JOIN DE AVGS POR TP
df = df.merge(grouped, on = 'TP', how = 'left')
#REEMPLAZAR Age VACIOS POR SU VALOR EN COLUMNA avg
df['Age'].fillna(df['avg'], inplace=True)
df = df.drop(['avg'], axis = 1)
df = pd.get_dummies(df, columns = ['Sex', 'Embarked','TP','Title'])
X = df.loc[:, df.columns != 'Survived']
y = df.loc[:, 'Survived']

In [3]:
#SPLITTING DATA INTO THE TRAINING AND TEST SET
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [4]:
#FEATURE SCALLING
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train  = sc_X.fit_transform(X_train)
X_test  = sc_X.transform(X_test)

In [5]:
#CREATING THE NEURAL NETWORK

#INITIALIZING THE ANN
ann = tf.keras.models.Sequential() #INITIALIZE AS A SEQUENCE OF LAYERS

#ADDING THE INPUT LAYER AND THE FIRSR HIDDEN LAYER
#THE INPUT NEURONS ARE THE COLUMNS
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu')) #THIS ADDS HOW MANY HIDDEN NEURONS WE WANT, RELU IS THE RECTIFIER ACTIVATION FUNCTION

#ADDING THE SECOND HIDDEN LAYER
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

#ADDING THE OUTPUT LAYER
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) #I NEED A SIGMOID ACTIVATION FUNCTIONS, TO GET PROB OF BINARY OUTCOME, TO GET NOT ONLY 0 OR 1, THE PERCENTAGE TOO, IF WE HAVE NON BINARY OUTCOME WE USE softmax


In [6]:
#COMPILING THE ANN
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy',metrics = ['accuracy']) #THE OPTIMIZER WILL UPDATE THE WEIGHTS TO REDUCE DE LOSS, LOSS WILL COMPUTE THE DIFFERENCE BETWEEN PREDICTED AND REAL RESULTS, IF ITS A CATEGORICAL NOT BINARY WE INTRODUCE categorical_crossentropy

#TRAINING THE ANN ON THE TRAINING SET
ann.fit(X_train, y_train, batch_size = 32, epochs = 100) #COMPARE RESULTS IN BATCH HAS MORE PERFOMANCE THAN ONE BY ONE

Epoch 1/100
23/23 [==============================] - 0s 781us/step - loss: 0.6728 - accuracy: 0.6152
Epoch 2/100
23/23 [==============================] - 0s 694us/step - loss: 0.6205 - accuracy: 0.6376
Epoch 3/100
23/23 [==============================] - 0s 694us/step - loss: 0.5827 - accuracy: 0.6671
Epoch 4/100
23/23 [==============================] - 0s 780us/step - loss: 0.5564 - accuracy: 0.7289
Epoch 5/100
23/23 [==============================] - 0s 737us/step - loss: 0.5378 - accuracy: 0.7795
Epoch 6/100
23/23 [==============================] - 0s 780us/step - loss: 0.5252 - accuracy: 0.7893
Epoch 7/100
23/23 [==============================] - 0s 781us/step - loss: 0.5160 - accuracy: 0.7935
Epoch 8/100
23/23 [==============================] - 0s 737us/step - loss: 0.5081 - accuracy: 0.7949
Epoch 9/100
23/23 [==============================] - 0s 824us/step - loss: 0.5012 - accuracy: 0.7949
Epoch 10/100
23/23 [==============================] - 0s 737us/step - loss: 0.4902 - accura

<class 'pandas.core.frame.DataFrame'>


NameError: name 'le' is not defined

In [8]:
#MAKING THE CONFUSION MATRIX
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
acs = accuracy_score(y_test, y_pred)
print(acs)

[[99 11]
 [22 47]]
0.8156424581005587


NameError: name 'classifier' is not defined